In [1]:
import torch
from torchvision.datasets import CelebA
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
import numpy as np
from typing import List, Tuple
import pickle
import time
from torch.utils.data import ConcatDataset, Dataset, Subset
%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]

In [3]:
# train_dataset = CelebA('D:/downloads/btp/outputforpaper/CELEBA', split="train", download=False, transform=transforms.ToTensor())
# test_dataset = CelebA('D:/downloads/btp/outputforpaper/CELEBA', split="test", download=False, transform=transforms.ToTensor())
train_dataset = CelebA('D:/downloads/btp/outputforpaper/CELEBA', split="train", download=True, transform=transforms.ToTensor())
test_dataset = CelebA('D:/downloads/btp/outputforpaper/CELEBA', split="test", download=True, transform=transforms.ToTensor())

2422.0 bytes
/home/vishal1/miniconda3/envs/nitinmtp/lib/python3.9/site-packages/torchvision/datasets/utils.py:260: UserWarning: We detected some HTML elements in the downloaded file. This most likely means that the download triggered an unhandled API response by GDrive. Please report this to torchvision at https://github.com/pytorch/vision/issues including the response:

<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="QQb_p0HGjtdFFmG9hGyzJw">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial,sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-alig

RuntimeError: The MD5 checksum of the download file D:/downloads/btp/outputforpaper/CELEBA/celeba/img_align_celeba.zip does not match the one on record.Please delete the file and try again. If the issue persists, please report this to torchvision at https://github.com/pytorch/vision/issues.

In [ ]:
total_train_size = len(train_dataset)
total_test_size = len(test_dataset)
input_dim = 12288
num_clients = 50
rounds = 50
batch_size = 16
in_group_rounds = 5 
epochs_per_client = 5
learning_rate = 5e-3 
num_malicious_clients = 10 

In [ ]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(DataLoader):
        def __init__(self, dl, device):
            self.dl = dl
            self.device = device

        def __iter__(self):
            for batch in self.dl:
                yield to_device(batch, self.device)

        def __len__(self):
            return len(self.dl)

device = get_device()


def average_models(new_model, next, scale):
    if new_model == None:
        new_model = next
        for key in new_model:
            new_model[key] = new_model[key] * scale
    else:
        for key in new_model:
            new_model[key] = new_model[key] + (next[key] * scale)
    return new_model

In [ ]:
class FederatedNet(nn.Module):    
    def __init__(self):
        super().__init__()
        
        # First convolution block (64x64 → 32x32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Use 3x3 kernel
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),  # LeakyReLU instead of ReLU
            nn.Conv2d(32, 32, kernel_size=3, padding=1),  # Extra conv layer
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2)  # Downsample 64x64 → 32x32
        )
        
        # Second convolution block (32x32 → 16x16)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Extra conv layer
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2)  # Downsample 32x32 → 16x16
        )

        # Third convolution block (16x16 → 8x8)
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # Extra conv layer
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2)  # Downsample 16x16 → 8x8
        )

        # Fourth convolution block (8x8 → 4x4)
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Extra conv layer
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2)  # Downsample 8x8 → 4x4
        )

        # Global Average Pooling + Fully Connected Layer
        self.gap = nn.AdaptiveAvgPool2d(1)  # Reduces 4x4x256 → 1x1x256
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),  # Reduce to 128 features
            nn.ReLU(),
            nn.Dropout(0.4),  # Dropout to prevent overfitting
            nn.Linear(128, 40)  # Output 40 attributes for CelebA
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)  # Global Average Pooling
        out = self.fc(out)
        return out  # No sigmoid (BCEWithLogitsLoss applies it internally)
    
    def batch_accuracy(self, outputs, labels):
        with torch.no_grad():
            predictions = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions
            correct = (predictions == labels).float().mean()  # Average over all attributes
            return correct
    
    def _process_batch(self, batch):
        images, labels = batch
        outputs = self(images)
        loss = nn.BCEWithLogitsLoss()(outputs, labels.float())  # Use BCE loss for multi-label classification
        accuracy = self.batch_accuracy(outputs, labels)
        return loss, accuracy
    
    def fit(self, dataset, epochs, lr, batch_size=128, opt=torch.optim.Adam):
        dataloader = DataLoader(dataset, batch_size, shuffle=True)
        optimizer = opt(self.parameters(), lr)
        history = []
        for epoch in range(epochs):
            losses = []
            accs = []
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss.detach()
                losses.append(loss)
                accs.append(acc)
            avg_loss = torch.stack(losses).mean().item()
            avg_acc = torch.stack(accs).mean().item()
            history.append((avg_loss, avg_acc))
        return history
    
    def evaluate(self, dataset, batch_size=128):
        dataloader = DataLoader(dataset, batch_size)
        losses = []
        accs = []
        with torch.no_grad():
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                losses.append(loss)
                accs.append(acc)
        avg_loss = torch.stack(losses).mean().item()
        avg_acc = torch.stack(accs).mean().item()
        return (avg_loss, avg_acc)

In [ ]:
def partition_data_label_quantity(
    num_clients, labels_per_client, train_dataset,total_train_size,seed=42):
    trainset=train_dataset
    prng = np.random.default_rng(seed)

    targets = trainset.targets
    if isinstance(targets, list):
        targets = np.array(targets)
    if isinstance(targets, torch.Tensor):
        targets = targets.numpy()
    num_classes = len(set(targets))
    times = [0 for _ in range(num_classes)]
    contains = []

    for i in range(num_clients):
        current = [i % num_classes]
        times[i % num_classes] += 1
        j = 1
        while j < labels_per_client:
            index = prng.choice(num_classes, 1)[0]
            if index not in current:
                current.append(index)
                times[index] += 1
                j += 1
        contains.append(current)
    idx_clients: List[List] = [[] for _ in range(num_clients)]
    for i in range(num_classes):
        idx_k = np.where(targets == i)[0]
        prng.shuffle(idx_k)
        idx_k_split = np.array_split(idx_k, times[i])
        ids = 0
        for j in range(num_clients):
            if i in contains[j]:
                idx_clients[j] += idx_k_split[ids].tolist()
                ids += 1
    trainsets_per_client = [Subset(trainset, idxs) for idxs in idx_clients]
    return trainsets_per_client

In [ ]:
client_datasets = partition_data_label_quantity(num_clients,2,train_dataset,len(train_dataset))

In [ ]:
import numpy as np


class memory:

    def __init__(self, batch_number, batch_size):
        self.batch_number = batch_number
        self.batch_size = batch_size
        self.matrix = np.zeros((batch_number, batch_size), dtype=int)
        self.add_time = 0

    def add_number(self, batch_no, true_array):
        if batch_no >= self.batch_number:
            print(f"Warning: Batch number {batch_no} out of range. Skipping this entry.")
            return
        # Ensure true_array length matches batch_size by padding or truncating as necessary
        true_array = np.pad(true_array, (0, max(0, self.batch_size - len(true_array))), mode='constant')[:self.batch_size]
        self.matrix[batch_no] += true_array


    def add(self):
        self.add_time = self.add_time + 1

    def get_ambiguous(self, number):
        temp_matrix = self.matrix
        temp_matrix = np.abs(temp_matrix - np.ones(self.matrix.shape) * self.add_time * 0.5)
        temp_array = np.reshape(temp_matrix, self.matrix.shape[0] * self.matrix.shape[1])
        temp = list(map(list, zip(range(len(temp_array)), temp_array)))
        small = sorted(temp, key=lambda x: x[1], reverse=False)
        small_array = []
        for i in range(number):
            small_array.append(small[i][0])
        return small_array

In [ ]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader

class KMT:
    learning_rate = 0.01
    momentum = 0.5
    batch_size = 32
    batch_number = 1000
    train_loss = []
    test_loss = []
    test_accuracy = []

    def __init__(self, dataset, index, test_loader):
        self.load_train_set(dataset, index)
        self.test_loader = test_loader
        self.model = FederatedNet()  # Using CelebA-compatible model
        self.criterion = nn.BCEWithLogitsLoss()  # Change for multi-label classification
        self.distillation_criterion = nn.KLDivLoss(reduction='batchmean')
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate, momentum=self.momentum)

    def load_train_set(self, dataset, index):
        train_dataset = torch.utils.data.Subset(dataset, index.astype(int))
        self.train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

    def train(self):
        running_loss = 0
        batch_number = 0

        for images, labels in self.train_loader:
            self.optimizer.zero_grad()
            output = self.model(images)
            loss = self.criterion(output, labels.float())  # Ensure labels are float for BCE

            # Multi-label accuracy computation
            predictions = (torch.sigmoid(output) > 0.5).int()
            equals = (predictions == labels).cpu().numpy().astype(int)

            # Backpropagation
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
            batch_number += 1

        self.train_loss.append(running_loss / len(self.train_loader))

    def distillation_train(self, images, labels, other_soft_decision):
        self.optimizer.zero_grad()
        output = self.model(images)
        loss1 = self.criterion(output, labels.float())
        loss2 = self.distillation_criterion(torch.sigmoid(output), other_soft_decision)
        loss = loss1 + loss2

        predictions = (torch.sigmoid(output) > 0.5).int()
        equals = (predictions == labels).cpu().numpy().astype(int)

        loss.backward()
        self.optimizer.step()
        return equals

    def test(self):
        accuracy = 0
        loss = 0
        for images, labels in self.test_loader:
            output = self.model(images)
            loss += self.criterion(output, labels.float())

            predictions = (torch.sigmoid(output) > 0.5).int()
            equals = (predictions == labels).float()
            accuracy += torch.mean(equals)

        self.test_loss.append((loss / len(self.test_loader)).item())
        self.test_accuracy.append((accuracy / len(self.test_loader)).item())
        return accuracy / len(self.test_loader)

    def get_ambiguous_data(self, array):
        np.sort(array)
        batch_number = 0
        ambiguous_images = []
        ambiguous_labels = []
        for images, labels in self.train_loader:
            for no in array:
                if int(no / self.batch_size) == batch_number:
                    column = np.mod(no, self.batch_size)
                    ambiguous_images.append(images[column])
                    ambiguous_labels.append(labels[column])
            batch_number += 1
        ambiguous_images = torch.stack(ambiguous_images)
        ambiguous_labels = torch.stack(ambiguous_labels)
        return ambiguous_images, ambiguous_labels


DFL

In [ ]:
class Client:
    def __init__(self, client_id, dataset, test_loader):
        self.client_id = client_id
        self.dataset = dataset
        # self.test_loader = test_loader
        self.kmt = KMT(dataset, index=np.arange(len(dataset)), test_loader=test_loader)
        self.client_data = []

    def get_dataset_size(self):
        return len(self.dataset)
    
    def get_client_id(self):
        return self.client_id

    def train(self, parameters_dict, other_soft_decisions=None):
        # Load initial parameters
        self.kmt.model.load_state_dict(parameters_dict)

        # Standard training
        self.kmt.train()

        # Knowledge distillation using soft labels from other clients (if available)
        if other_soft_decisions is not None:
            for images, labels in self.kmt.train_loader:
                if images in other_soft_decisions:
                    other_soft_decision = other_soft_decisions[images]
                    self.kmt.distillation_train(images, labels, other_soft_decision)
        
        # Return the updated model parameters
        return self.kmt.model.state_dict()

    def recieve_parameters(self, client_id, client_parameter, fraction):
        client_object = {"client_id": client_id, "client_parameter": client_parameter, "fraction": fraction}
        self.client_data.append(client_object)

    def aggregated_parameters(self):
        local_aggregated_model = None
        for i in range(len(self.client_data)):
            local_aggregated_model = average_models(local_aggregated_model, self.client_data[i]["client_parameter"], self.client_data[i]["fraction"])
        self.client_data.clear()
        return local_aggregated_model


In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)
clients = [Client(i, client_datasets[i], test_loader) for i in range(num_clients)]

In [ ]:
dfl_start_time = time.time()

global_net = to_device(FederatedNet(), device)
curr_parameters = global_net.state_dict()
history_dfl = []
weights = []
traffic = []
load_model = None
total_data_size = len(train_dataset)

for i in range(rounds):
    new_model = None
    soft_decisions = {}  # Dictionary to store each client's soft decisions for distillation

    # Step 1: Each client performs training and generates soft decisions
    for client in clients:
        fraction = client.get_dataset_size() / total_data_size

        # Each client trains on its dataset, possibly using other clients' soft decisions for distillation
        client_parameters = client.train(curr_parameters, other_soft_decisions=soft_decisions if i > 0 else None)
        
        # Gather soft decisions for this client's data to use in distillation
        for images, labels in client.kmt.train_loader:
            output = client.kmt.model(images)
            soft_decisions[images] = torch.softmax(output, dim=1).detach()

        # Share parameters with other clients for aggregation
        for clt in clients:
            clt.recieve_parameters(client.get_client_id(), client_parameters, fraction)

        # Aggregate models across all clients
        new_model = average_models(new_model, client_parameters, fraction)

    # Update the global model and evaluate
    load_model = new_model.copy()
    global_net.load_state_dict(new_model)
    train_loss, train_acc = global_net.evaluate(train_dataset)
    dev_loss, dev_acc = global_net.evaluate(test_dataset)
    
    # Logging
    print(f'After round {i + 1}, train_loss = {round(train_loss, 4)}, dev_loss = {round(dev_loss, 4)}, dev_acc = {round(dev_acc, 4)}')
    history_dfl.append((train_acc, dev_acc, i))
    traffic.append((num_clients) * (num_clients - 1))

# Calculate total time
dfl_end_time = time.time()
dfl_time = dfl_end_time - dfl_start_time
print(f"Total training time: {dfl_time:.2f} seconds")

# Optionally save the final global model if needed
torch.save(global_net.state_dict(), "final_global_model.pt")

In [ ]:
plt.plot([i + 1 for i in range(len(history_dfl))], [history_dfl[i][0] for i in range(len(history_dfl))], color='r', label='train accuracy')
plt.plot([i + 1 for i in range(len(history_dfl))], [history_dfl[i][1] for i in range(len(history_dfl))], color='b', label='dev accuracy')
plt.legend()
plt.title("Rounds vs Accuracy")
plt.xlabel("Number of rounds")
plt.ylabel("Accuracy")
plt.show()

HDFL

In [ ]:
class Client:
    def __init__(self, client_id, dataset, test_loader):
        self.client_id = client_id
        self.dataset = dataset
        self.kmt = KMT(dataset, index=np.arange(len(dataset)), test_loader=test_loader)
        self.client_data=[]
        self.server_id=-1
        self.distance=random.randint(1,100)

    def set_server_id(self,client_id):
        self.client_data_server=[]
        self.server_id=client_id
    
    def get_server_id(self):
        return self.server_id

    def get_dataset_size(self):
        return len(self.dataset)
    
    def get_client_id(self):
        return self.client_id

    def train(self, parameters_dict, other_soft_decisions=None):
        # Load initial parameters
        self.kmt.model.load_state_dict(parameters_dict)

        # Standard training
        self.kmt.train()

        # Knowledge distillation using soft labels from other clients (if available)
        if other_soft_decisions is not None:
            for images, labels in self.kmt.train_loader:
                if images in other_soft_decisions:
                    other_soft_decision = other_soft_decisions[images]
                    self.kmt.distillation_train(images, labels, other_soft_decision)
        
        # Return the updated model parameters
        return self.kmt.model.state_dict()

    def recieve_parameters(self, client_id, client_parameter, fraction):
        client_object = {"client_id": client_id, "client_parameter": client_parameter, "fraction": fraction}
        self.client_data.append(client_object)

    def aggregated_parameters(self):
        local_aggregated_model = None
        for i in range(len(self.client_data)):
            local_aggregated_model = average_models(local_aggregated_model, self.client_data[i]["client_parameter"], self.client_data[i]["fraction"])
        self.client_data.clear()
        return local_aggregated_model
    
    def recieve_data_as_server(self,client_parameter,fraction):
        client_object={"client_parameter":client_parameter,"fraction":fraction}
        self.client_data_server.append(client_object)
    

    def aggregate_parameter_as_server(self):
         local_aggregated_model_as_server=None
         for data in self.client_data_server:
              local_aggregated_model_as_server=average_models(local_aggregated_model_as_server,data["client_parameter"],data["fraction"])
         self.client_data_server.clear()
         return local_aggregated_model_as_server


In [ ]:
def utility_function(client_set_1,client_set_2,data):
  client_set_1_average=None
  client_set_2_average=None
  for i in client_set_1:
    client_set_1_average=average_models(client_set_1_average,data[i],1)
  for i in client_set_2:
    client_set_2_average=average_models(client_set_2_average,data[i],1)
  model1_weights = []
  model2_weights = []
  for key, value in client_set_1_average.items():
      model1_weights.append(value.detach().cpu())

  for key, value in client_set_2_average.items():
      model2_weights.append(value.detach().cpu())
  weight1_flat = torch.cat([param.view(-1) for param in model1_weights])
  weight2_flat = torch.cat([param.view(-1) for param in model2_weights])
  cosine_sim = torch.nn.functional.cosine_similarity(weight1_flat, weight2_flat, dim=0)
  dissimilarity_score = 1 - cosine_sim.item()
  return dissimilarity_score

In [ ]:
def generateGroupsGame(data,num_clients):
    groups=[]
    for i in range(0,num_clients):
        groups.append([i])
    while True:
        for group in groups:
            for x in group:
                no_change_flag=0
                max_group=None
                max_utility=0
                for g in groups:
                    if g!=group:
                        if len(g) >= 1 and len(group) > 1:
                            tempset_1=g.copy()
                            tempset_1.append(x)
                            tempset_2=group.copy()
                            tempset_2.remove(x)
                            if utility_function(tempset_1,tempset_2,data) > utility_function(g,group,data): 
                                if max_utility < utility_function(tempset_1,tempset_2,data):
                                    max_group=g
                                    max_utility=utility_function(tempset_1,tempset_2,data)        
                        elif len(group)<=1:
                            if utility_function(g,group,data) > max_utility:
                                max_group=g
                                max_utility=utility_function(g,group,data)
                if max_group is not None:  
                    group.remove(x)
                    max_group.append(x)
                    no_change_flag=1
                    if len(group) <=0 :
                        groups.remove(group)
        if no_change_flag == 0:
            break
    print("end")
    return groups

In [ ]:
def groupDataLength(groups,clients):
    group_dataset_length=[]
    for grp in groups:
        totalcount=0
        for clt in grp:
            totalcount+=clients[clt].get_dataset_size()
        group_dataset_length.append(totalcount)
    return group_dataset_length

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)
clients = [Client(i, client_datasets[i], test_loader) for i in range(num_clients)]

In [ ]:
global_net = to_device(FederatedNet(), device)
curr_parameters = global_net.state_dict()
data = []
for client in clients:
    paramerter=client.train(curr_parameters)
    data.append(paramerter)

groups=generateGroupsGame(data,num_clients)
print(groups)
group_dataset_length=groupDataLength(groups,clients)

In [ ]:
hdfl_start_time = time.time()

global_net = to_device(FederatedNet(), device)
curr_parameters = global_net.state_dict()
history_hdfl = []
weights = []
traffic = []
load_model = None
total_data_size = len(train_dataset)

for i in range(rounds):
    new_model = None
    soft_decisions = {}  # Dictionary to store each client's soft decisions for distillation

    # Step 1: Each client performs training and generates soft decisions
    for client in clients:
        fraction = client.get_dataset_size() / total_data_size

        # Each client trains on its dataset, possibly using other clients' soft decisions for distillation
        client_parameters = client.train(curr_parameters, other_soft_decisions=soft_decisions if i > 0 else None)
        
        # Gather soft decisions for this client's data to use in distillation
        for images, labels in client.kmt.train_loader:
            output = client.kmt.model(images)
            soft_decisions[images] = torch.softmax(output, dim=1).detach()

        # Share parameters with other clients for aggregation
        for clt in clients:
            clt.recieve_parameters(client.get_client_id(), client_parameters, fraction)

        # Aggregate models across all clients
        new_model = average_models(new_model, client_parameters, fraction)

    # Update the global model and evaluate
    load_model = new_model.copy()
    global_net.load_state_dict(new_model)
    train_loss, train_acc = global_net.evaluate(train_dataset)
    dev_loss, dev_acc = global_net.evaluate(test_dataset)
    
    # Logging
    print(f'After round {i + 1}, train_loss = {round(train_loss, 4)}, dev_loss = {round(dev_loss, 4)}, dev_acc = {round(dev_acc, 4)}')
    history_hdfl.append((train_acc, dev_acc, i))
    traffic.append((num_clients) * (num_clients - 1))

# Calculate total time
hdfl_end_time = time.time()
hdfl_time = hdfl_end_time - hdfl_start_time
print(f"Total training time: {hdfl_time:.2f} seconds")

# Optionally save the final global model if needed
torch.save(global_net.state_dict(), "final_global_model.pt")


In [ ]:
plt.plot([i + 1 for i in range(len(history_hdfl))], [history_hdfl[i][0] for i in range(len(history_hdfl))], color='r', label='train accuracy')
plt.plot([i + 1 for i in range(len(history_hdfl))], [history_hdfl[i][1] for i in range(len(history_hdfl))], color='b', label='dev accuracy')
plt.legend()
plt.title("Rounds vs Accuracy")
plt.xlabel("Number of rounds")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
plt.plot([i + 1 for i in range(len(history_dfl))], [history_dfl[i][1] for i in range(len(history_dfl))], color='b', label='DFL dev accuracy')
plt.plot([i + 1 for i in range(len(history_hdfl))], [history_hdfl[i][1] for i in range(len(history_hdfl))], color='r', label='HDFL dev accuracy')
plt.legend()
plt.title("Rounds vs Accuracy on CIFAR10")
plt.xlabel("Number of rounds")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
labels = ['DFL', 'HDFL']
values = [dfl_time, hdfl_time]

# Plot the bar graph
plt.bar(labels, values, color=['green', 'red'])

# Add labels and title
plt.xlabel('')
plt.ylabel('RUN TIME')
plt.title('Run Time for DFL & HDFL')

# Show the plot
plt.show()